In this notebook, we empirically explore a surprising result which states that the dimension metric follows a non-monotonous zizag in Erdős–Rényi.

In [ ]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import sys
sys.path.append('../')
from helpers import *
import plotly.graph_objs as go

In [ ]:
n_ = [50, 300, 1000, 3000]
start = {50: 0.5, 300: 0.4, 1000: 0.29, 3000: 0.29} # Make sure the graphs are connected

In [ ]:
simulations_res = {}

for n in n_:
    solutions_temp = {}
    for x in tqdm(np.linspace(start[n], 0.95, 15)):

        G = nx.erdos_renyi_graph(n, n**(x - 1))
        length = dict(nx.all_pairs_shortest_path_length(G))
        node_list = list(G.nodes())
        nb_of_iters = 100

        high = n
        low = 0

        while low + 1 < high:
            middle = (high+low) // 2
            num_nodes = middle # Number of nodes to sample
            node_list = list(G.nodes())
            count = 0
            for i in range(nb_of_iters):
                nodes = set(random.sample(node_list, num_nodes)) # Random set of nodes to test
                if is_resolving_set(G, nodes, length):
                    count += 1
            v = count / nb_of_iters
            if (count / nb_of_iters) > 0.5:
                high = middle
                v_high = v 
            else:
                low = middle
                v_low = v
        if abs(v_low - 0.5) < abs(v_high - 0.5):
            solutions_temp[x] = low
        else:
            solutions_temp[x] = high
    simulations_res[n] = solutions_temp

In [ ]:
#with open('zigzag/zigzag_p={}_n={}.pickle'.format(0.5, n4), 'wb') as file:
    #pickle.dump(solutions4, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#with open('zigzag/zigzag_p={}_n={}.pickle'.format(0.5, n4), "rb") as file:
    #solutions4 = pickle.load(file)

In [ ]:
# Define your data
x1 = list(simulations_res[n_[0]].keys())
y1 = list(np.emath.logn(n_[0], list(simulations_res[n_[0]].values())/np.log(n_[0])))

x2 = list(simulations_res[n_[1]].keys())
y2 = list(np.emath.logn(n_[1], list(simulations_res[n_[1]].values())/np.log(n_[1])))

x3 = list(simulations_res[n_[2]].keys())
y3 = list(np.emath.logn(n_[2], list(simulations_res[n_[2]].values())/np.log(n_[2])))

x4 = list(simulations_res[n_[3]].keys())
y4 = list(np.emath.logn(n_[3], list(simulations_res[n_[3]].values())/np.log(n_[3])))

# The value it converges to
x_t = np.linspace(0.01, 1, 10000)
y_t = [1 - xi*np.floor(1/xi) for xi in x_t]

# Define the trace for the scatter plot
trace_1 = go.Scatter(x=x1, y=y1, mode='markers+lines',name='Simulations with n={}'.format(n_[0]))
trace_2 = go.Scatter(x=x2, y=y2, mode='markers+lines',name='Simulations with n={}'.format(n_[1]))
trace_3 = go.Scatter(x=x3, y=y3, mode='markers+lines',name='Simulations with n={}'.format(n3))
trace_4 = go.Scatter(x=x4, y=y4, mode='markers+lines',name='Simulations with n={}'.format(n4))


trace_t = go.Scatter(x=x_t, y=y_t, mode='markers', marker=dict(size=2), name='Theoretical value')

# Define the layout
layout = go.Layout(title=r"$\text{Behaviour of } \log_n\frac{\beta(G(n,\ n^{x-1}))}{\log n} \text{ as a function of } x$", 
                   title_x=0.5,
                   xaxis=dict(title='$x$'), 
                   yaxis=dict(title=r'$\log_n \frac{\beta(G(n, \ n^{x-1}))}{\log n}$'),
                   legend=dict(x=0.05, y=0.9, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace_1, trace_2, trace_t], layout=layout)

# Show the figure
fig.show()